### Two transmon optimisation
Two coupled transmons with five energy levels each. Both store two qubits. The generation creates a superposition of 2 LOs and envelopes per transmon. This file is used for optimising entangling gates between the transmons.

In [ ]:
import argparse
import copy
import os
import sys

# Libs and helpers
from collections import OrderedDict

import numpy as np
import scipy.linalg
import tensorflow as tf

import c3.libraries.algorithms as algorithms
import c3.libraries.envelopes
import c3.libraries.fidelities as fidelities
import c3.utils.qt_utils as qt_utils
from c3.signal.pulse import EnvelopeDrag
from c3.utils.tf_utils import tf_project_to_comp, tf_abs, tf_unitary_overlap
import four_level_transmons.custom_gates as custom_gates
from c3.experiment import Experiment as Exp
# Main C3 objects
from c3.libraries import constants
from c3.model import Model as Mdl
from c3.optimizers.optimalcontrol import OptimalControl
from c3.parametermap import ParameterMap as PMap
from four_level_transmons.DataOutput import DataOutput
from four_level_transmons.custom_envelopes import *
from four_level_transmons.plotting import *
from four_level_transmons.utilities import *
from four_level_transmons.blackbox import generateSignalFromConfig
from four_level_transmons.notebook_utils import *

tf.config.run_functions_eagerly(True)
np.set_printoptions(linewidth=300)

In [ ]:
if len(sys.argv[1:]) > 0 and "ipykernel_launcher" not in sys.argv[0]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--output", help="Output directory")
    args = parser.parse_args()
    output_dir = args.output
    print("Output directory: ", output_dir)
else:
    print("=========== WARNING: no output directory specified ============")
    output_dir = "./output"
output = DataOutput(output_dir, file_suffix='before')

In [ ]:
# general settings
numPWCPieces = 60
usePWC = False
useDRAG = False
t_final = 1000e-9
sim_res = 20e9
awg_res = numPWCPieces / t_final if usePWC else 2e9
isDressed = False
isRotating = False

ALGORITHM_LBFGS = 0
ALGORITHM_LBFGS_GRAD_FREE = 1
ALGORITHM_CMAES = 2
ALGORITHM_GCMAES = 3
selected_algorithms = [
    #(ALGORITHM_LBFGS, {"maxfun": 50}),
    #(ALGORITHM_CMAES, {}),
    (ALGORITHM_LBFGS, {"maxfun": 1500, "ftol": 2e-6})
]

In [ ]:
IDEAL_GATE, IDEAL_GATE_NAME = np.eye(16), "unity"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_CNOT_t1q2_t2q2, "cnot_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_iCNOT_t1q2_t2q2, "icnot_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_iSWAP_t1q2_t2q2, "iswap_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_SQRTiSWAP_t1q2_t2q2, "sqrtiswap_t1q2_t2q2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_CZ_t1q2_t2q2, "cz_t1_t2"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_UNIVERSAL_ENTANGLER, "universal_entangler"
#IDEAL_GATE, IDEAL_GATE_NAME = custom_gates.GATE_4QUBIT_1110_1111, "cccnot"

OPTIMISE_TFINAL = False
OPTIMISE_FREQUENCIES = True

In [ ]:
# coupling=30: f=4.6, anh=-200; f=4.32, anh=-350
# f=4.5: anh=-200, coupling=33
# Initialise the qubits and drive lines
qubit_levels = [5, 5]
qubit_frequencies = [5e9, 4.5e9]
anharmonicities = [-300e6, -250e6]
t1s = [25e-6, 25e-6]
t2stars = [35e-6, 35e-6]
qubit_temps = 50e-3
couplingStrength = 20e6
print("qubits frequencies: ", qubit_frequencies, "anharmonicities: ", anharmonicities,
      "coupling: ", couplingStrength)

level_labels_transmon = ["|0,0\\rangle", "|0,1\\rangle", "|1,0\\rangle", "|1,1\\rangle"]
for i in range(len(level_labels_transmon), max(qubit_levels)):
    level_labels_transmon.append("leakage")
level_labels = []
level_labels_with_leakage = []
level_labels_short = []
for i in range(qubit_levels[0]):
    for j in range(qubit_levels[1]):
        if i > 3 or j > 3:
            level_labels_with_leakage.append("leakage")
            level_labels_short.append(None)
        else:
            s = f"${level_labels_transmon[i]},{level_labels_transmon[j]}$"
            level_labels.append(s)
            level_labels_with_leakage.append(s)
            level_labels_short.append(f"{i},{j}")
level_labels_transmon = [f"${x}$" for x in level_labels_transmon]

qubits = createQubits(qubit_levels, qubit_frequencies, anharmonicities,
                         t1s, t2stars, qubit_temps)
coupling = createChainCouplings([couplingStrength], qubits)
drives = createDrives(qubits)

In [ ]:
# Create the model
model = Mdl(qubits, coupling + drives)
model.set_lindbladian(False)
model.set_dressed(isDressed)
model.set_FR(isRotating)

#energies = model.get_Hamiltonian().numpy().diagonal().real / (2 * np.pi)
#print("energies: ", energies)
qubitEnergies = [q.get_Hamiltonian().numpy().diagonal().real / (2 * np.pi) for q in qubits]
qubitEnergies[0] = qubitEnergies[0][::qubit_levels[0]]
qubitEnergies[1] = qubitEnergies[1][:qubit_levels[1]]
qubitTransitions = [np.array([e[i + 1] - e[i] for i in range(len(e) - 1)]) for e in qubitEnergies]
for i in range(len(qubits)):
    print(f"Qubit {i}:")
    print(qubitEnergies[i])
    print(qubitTransitions[i])

In [ ]:
'''
stored_pmap = PMap()
#stored_pmap.read_config("./optimised_params/2transmons_uncoupled/Z_q2_200ns.json")
stored_pmap.read_config("./optimised_params/CZ_f.json")
stored_params = stored_pmap.asdict()["CZ_t1_t2[0, 1]"]

stored_signal = generateSignalFromConfig(stored_params, sim_res=sim_res, awg_res=awg_res, useDRAG=useDRAG,
                                         usePWC=usePWC, numPWCPieces=numPWCPieces, t_final=t_final)
print(stored_signal["d1"]["values"].shape)
'''

In [ ]:
'''
stored_pmap = PMap()
stored_pmap.read_config("./optimised_params/fourier basis 5 levels/17freqs 1000ns/unity.json")
stored_params = stored_pmap.asdict()[list(stored_pmap.asdict().keys())[0]]

carrier_freqs = []
pulse_xy_angles = []
pulse_amps = []
ampFactors = {"d1": 33, "d2": 0.8}
for driveName, drive in stored_params["drive_channels"].items():
    env = drive[f"envelope_{driveName}"]
    #print(env)
    carrier_freqs.append(env.params["freqs"].get_value().numpy() / (2 * np.pi))
    pulse_xy_angles.append(env.params["phases"].get_value().numpy())
    pulse_amps.append(ampFactors[driveName] * env.params["amps"].get_value().numpy()) # env.params["amp"].get_value().numpy()
'''

In [ ]:
# Create the generator
'''
cos_frequencies = np.array([4995997406.919341, 4704950761.420654, 4701014384.994876, 5007032274.618064,
                            4062167078.4774194, 4148445305.307898, 4402001629.33434, 4245160832.138902,
                            5000004596.914059, 4715989016.545379, 4377994950.489334, 4691989633.233531,
                            4110695567.278656, 4106672258.1941147])
cos_amplitudes = np.array([0.006776392936855479, 0.006183858694756216, 0.005921233065898419, 0.0053621575456050865,
                           0.0058839116357663175, 0.0041460859937105295, 0.0040101476207611226, 0.003769561777275727,
                           0.003031120830568577, 0.0031130260335161517, 0.002403971046868337, 0.002353206535976198,
                           0.002012669743665184, 0.001825629728895534])
cos_phases = np.array([1.6381027349231951, 2.5838889632467543, -0.6247481636275114, -2.6722366011197876,
                       2.8325504483054322, 1.9886986099203705, 3.1009812892045643, -0.9957720554988438,
                       1.2472118040300213, -1.2076080330207344, -2.911563673846197, -1.4505219481308833,
                       2.4095913176783403, -1.1073120929779088])
'''
generator = createGenerator(drives, sim_res=sim_res, awg_res=awg_res)

In [ ]:
# Envelopes and carriers
'''
carrier_freqs = [
    #[0.5*(qubitTransitions[0][1]+qubitTransitions[0][0]), 0.5*(qubitTransitions[0][2]+qubitTransitions[0][1])],
    #[qubitTransitions[1][0], qubitTransitions[1][1]],
    [40e6, 563e6],
    #[qubitTransitions[1][0]],
    #[0.5*(qubitTransitions[1][1]+qubitTransitions[1][0]), 0.5*(qubitTransitions[1][2]+qubitTransitions[1][1])],
    #[qubitTransitions[1][0], qubitTransitions[1][1]]
    #[qubitTransitions[1][0]]
    [121e6, 644e6]
]
#df = couplingStrength ** 2 / (qubit_frequencies[0] - qubit_frequencies[1])
#print("shift: ", df)
#carrier_freqs[0] = [x + df for x in carrier_freqs[0]]
#carrier_freqs[1] = [x - df for x in carrier_freqs[1]]

carrier_framechange = [
    [0.01, 0.01],
    [0.01, 0.01]
]
pulse_t_final = [
    [t_final, t_final],
    [t_final, t_final]
]
pulse_sigmas = [
    [t_final/5, t_final/5],
    [t_final/5, t_final/5]
]
pulse_amps = [
    [6, 6],
    [0.1, 0.1]
]
pulse_deltas = [
    [0.01, 0.01],
    [0.01, 0.01]
]
pulse_xy_angles = [
    [0.01, 0.01],
    [0.01, 0.01]
]
pulse_freq_offsets = [
    [0.01, 0.01],
    [0.01, 0.01]
]
'''
carrier_freqs = [[5e9], [4e9]]
pulse_xy_angles = [[1e-6], [1e-6]]
pulse_amps = [[0.001], [0.001]]
carrier_framechange = [[0.0], [0.0]]
pulse_freq_offsets = [[0.0], [0.0]]
pulse_t_final = [[1000e9], [1000e9]]
pulse_sigmas = [[1000e9], [1000e9]]
pulse_deltas = [[0.0], [0.0]]

envelopes = []
envelopesForDrive = {d.name: [] for d in drives}
carriers = []
carriersForDrive = {d.name: [] for d in drives}

# copy carriers and envelopes from file
for idx in []:
    dstIdx = idx
    srcIdx = idx
    driveNameSrc = drives[srcIdx].name
    driveNameDst = drives[dstIdx].name
    stored_params_d = stored_params["drive_channels"][driveNameSrc]
    for i in range(0, len(carrier_freqs[idx])):
        env: pulse.Envelope = copy.deepcopy(stored_params_d[f"envelope_{driveNameSrc}_{i + 1}"])
        if useDRAG and not isinstance(env, EnvelopeDrag):
            env = convertToDRAG(env)
        if usePWC:
            if env.shape != c3.libraries.envelopes.pwc:
                env = convertToPWC(env, numPWCPieces)
            elif len(env.params["inphase"]) != numPWCPieces:
                env = resamplePWC(env, numPWCPieces)
        env.name = f"envelope_{driveNameDst}_{i + 1}"
        #env = scaleGaussianEnvelope(env, t_final / env.params['t_final'].get_value())
        #env.params["amp"] = scaleQuantity(env.params["amp"], 5)
        #if dstIdx==1:
        #    env.params["amp"] = scaleQuantity(env.params["amp"], 0.01)
        envelopes.append(env)
        envelopesForDrive[driveNameDst].append(env)

        #shift = df if dstIdx == 0 else -df
        carrier = copy.deepcopy(stored_params_d[f"carrier_{driveNameSrc}_{i + 1}"])
        #print("Frequency: ", carrier.params["freq"].get_value() / (2 * np.pi),
        #      carrier.params["freq"].get_limits()[0] / (2 * np.pi),
        #      carrier.params["freq"].get_limits()[1] / (2 * np.pi))
        #print(stored_params_d[f"carrier_{driveNameSrc}_{i + 1}"])
        #carrier.params["freq"].set_value(
        #    carrier.params["freq"].get_value() / (2 * np.pi) + shift
        #)
        #carrier.params["freq"] = Qty(
        #    value=carrier_freqs[dstIdx][i],
        #    min_val=0.95 * carrier_freqs[dstIdx][i],
        #    max_val=1.05 * carrier_freqs[dstIdx][i],
        #    unit="Hz 2pi"
        #)
        carrier.name = f"carrier_{driveNameDst}_{i + 1}"
        carriers.append(carrier)
        carriersForDrive[driveNameDst].append(carrier)

# create carriers and envelopes
for idx in [0, 1]:
    for i in range(0, len(carrier_freqs[idx])):
        env = createGaussianPulse(
            t_final=pulse_t_final[idx][i],
            sigma=pulse_sigmas[idx][i],
            amp=pulse_amps[idx][i],
            delta=pulse_deltas[idx][i],
            xy_angle=pulse_xy_angles[idx][i],
            freq_off=pulse_freq_offsets[idx][i],
            useDrag=useDRAG
        )
        #env = createNoDriveEnvelope(t_final)
        if usePWC:
            env = convertToPWC(env, numPWCPieces)
        env.name = f"envelope_{drives[idx].name}_{i + 1}"
        #env.params["amp"] = scaleQuantity(env.params["amp"], 0.2)
        envelopes.append(env)
        envelopesForDrive[drives[idx].name].append(env)

        carrier_parameters = {
            "freq": Qty(value=carrier_freqs[idx][i], min_val=0.98 * carrier_freqs[idx][i],
                        max_val=1.02 * carrier_freqs[idx][i], unit="Hz 2pi"),
            "framechange": Qty(value=carrier_framechange[idx][i], min_val=-np.pi, max_val=3 * np.pi, unit="rad"),
        }
        carrier = pulse.Carrier(
            name=f"carrier_{drives[idx].name}_{i + 1}",
            desc="Frequency of the local oscillator",
            params=carrier_parameters,
        )
        carriers.append(carrier)
        carriersForDrive[drives[idx].name].append(carrier)

print("carrier: ", [[carrier.params["freq"] for carrier in carriers] for carriers in carriersForDrive.values()])
print("amp: ", [[env.params["amp"] for env in envelopes] for envelopes in envelopesForDrive.values()])

In [ ]:
printMatrix(IDEAL_GATE, level_labels, "ideal_gate", output)

gate = gates.Instruction(
    name=IDEAL_GATE_NAME,
    #name="unity",
    targets=[0, 1],
    t_start=0.0,
    t_end=t_final,
    channels=[d.name for d in drives],
    ideal=IDEAL_GATE,
)
for drive in drives:
    for env in envelopesForDrive[drive.name]:
        gate.add_component(copy.deepcopy(env), drive.name)
    for carrier in carriersForDrive[drive.name]:
        gate.add_component(copy.deepcopy(carrier), drive.name)

In [ ]:
# all energy levels with labels
stateEnergies = []
H = model.get_Hamiltonian().numpy()
evals,evecs = scipy.linalg.eig(H)
evals = evals.real / (2 * np.pi)
indices = [np.argmax(np.round(evecs[i], 2)) for i in range(len(evals))]
for i, x in enumerate(level_labels_short):
    if x is not None:
        energy = evals[indices[i]]
        stateEnergies.append((energy, x))

# all energy transitions
items = sorted(stateEnergies, key=lambda x: x[0])
transitions = []
for i in range(len(items)):
    for j in range(len(items)):
        if i != j:
            #print(i, j)
            E = items[j][0] - items[i][0]
            if E > 0:
                transitions.append((E, items[i][1] + " - " + items[j][1]))
#for t in transitions:
#    print(t)

#transitions.sort(key=lambda x: x[0])
#for t in transitions:
#    print(t)

directTransitions = np.array([np.abs(stateEnergies[i + 1][0] - stateEnergies[i][0]) for i in range(0, len(stateEnergies) - 1, 1)])
#directTransitions = []
#for i in range(0, len(stateEnergies)-1, 1):
#    diff = stateEnergies[i+1][0] - stateEnergies[i][0]
#    directTransitions.append(diff)
#print(directTransitions)

In [ ]:
# Set up the experiment
parameter_map = PMap(instructions=[gate], model=model, generator=generator)
exp = Exp(pmap=parameter_map, sim_res=sim_res)
exp.set_opt_gates([gate.get_key()])

unitaries = exp.compute_propagators()
printPropagator(exp, gate, level_labels_with_leakage, output)
#printAllSignals(exp, qubit, output, directory="devices_before")
printSignal(exp, qubits, gate, output=output, states=transitions)
#printMatrix(model.get_Hamiltonian(), level_labels_with_leakage, 'Hamiltonian', output)
#printMatrix(model.get_Hamiltonian() / np.max(model.get_Hamiltonian()), level_labels_with_leakage, 'Hamiltonian_scaled', output)

In [ ]:
# Specify the initial state
psi_init = [[0] * model.tot_dim]
#for i in entanglementInitStateFull:
#    psi_init[0][i] = 1
psi_init[0][0] = 1
psi_init /= np.linalg.norm(psi_init)
print("initial state: ", psi_init)
init_state = tf.transpose(tf.constant(psi_init, tf.complex128))
sequence = [gate.get_key()]

printTimeEvolution(exp, init_state, gate, level_labels, output)
#printEntanglementEvolution(exp, gate, output)
parameter_map.write_config(output.createFileName("parameter_map", "json"))

In [ ]:
# Specify the parameters to be optimised and initialise the optimiser
opt_map = []
for drive in drives:
    for env in envelopesForDrive[drive.name]:
        opt_map.append([(gate.get_key(), drive.name, env.name, "amp")])
        #opt_map.append([(gate.get_key(), drive.name, env.name, "freq_offset")])
        opt_map.append([(gate.get_key(), drive.name, env.name, "xy_angle")])
        if useDRAG:
            opt_map.append([(gate.get_key(), drive.name, env.name, "delta")])
        if usePWC:
            opt_map.append([(gate.get_key(), drive.name, env.name, "inphase")])
            opt_map.append([(gate.get_key(), drive.name, env.name, "quadrature")])
            #opt_map.append([(gate.get_key(), drive.name, env.name, "t_bin_end")])
        else:
            opt_map.append([(gate.get_key(), drive.name, env.name, "sigma")])
        if OPTIMISE_TFINAL:
            opt_map.append([(gate.get_key(), drive.name, env.name, "t_final")])
    for carrier in carriersForDrive[drive.name]:
        opt_map.append([(gate.get_key(), drive.name, carrier.name, "freq")])
    #    #opt_map.append([(gate.get_key(), drive.name, carrier.name, "framechange")])
parameter_map.set_opt_map(opt_map)
parameter_map.print_parameters()

In [ ]:
infidelities = []

for algorithm in selected_algorithms:
    if algorithm == ALGORITHM_LBFGS:
        infidelities = optimise(output, qubits, exp, algorithms.lbfgs, {
            "maxfun": 2000,
            "ftol": 1e-6
        }, gate)
    elif algorithm == ALGORITHM_LBFGS_GRAD_FREE:
        infidelities = optimise(output, qubits, exp, algorithms.lbfgs_grad_free, {
            "maxfun": 500,
            "gtol": 1e-4,
            "ftol": 1e-4
        }, gate)
    elif algorithm == ALGORITHM_CMAES:
        infidelities = optimise(output, qubits, exp, algorithms.cmaes, {
            "popsize": 15,
            "spread": 0.02,
            "maxfevals": 2000,
            "init_point": "True",
            "stop_at_sigma": 1e-3,
            "stop_at_convergence": 20
        }, gate)
    elif algorithm == ALGORITHM_GCMAES:
        infidelities = optimise(output, qubits, exp, algorithms.gcmaes, {
            "cmaes": {"popsize": 12, "spread": 0.05, "maxfevals": 20,
                      "init_point": "True", "stop_at_sigma": 1e-4, "stop_at_convergence": 20},
            "lbfgs": {"maxfun": 500, "ftol": 1e-6}
        }, gate)
    else:
        print("Unknown algorithm: ", algorithm)

In [ ]:
# Plot results
output = DataOutput(output_dir, file_suffix='after')
plotData(np.arange(len(infidelities)), infidelities, xlabel="Step",
         ylabel="Infidelity", filename=output.createFileName("convergence", "svg"))
printSignal(exp, qubits, gate, output=output, states=transitions)
#printAllSignals(exp, qubits, output, directory="devices_after")
printPropagator(exp, gate, level_labels_with_leakage, output, savePartials=False)
printTimeEvolution(exp, init_state, gate, level_labels, output)
#printEntanglementEvolution(exp, gate, output)
parameter_map.write_config(output.createFileName("parameter_map", "json"))

In [ ]:
'''
import numpy as np
from typing import Tuple
import tensorflow as tf

def makhlinInvariants(U: tf.Tensor) -> tf.Tensor:
    # transform to bell basis
    Q = tf.constant(np.matrix([
        [1, 0, 0, 1j],
        [0, 1j, 1, 0],
        [0, 1j, -1, 0],
        [1, 0, 0, -1j]
    ]) / np.sqrt(2))
    Ub = tf.matmul(tf.linalg.adjoint(Q), tf.matmul(U, Q))

    # calculate characteristics
    m = tf.matmul(tf.transpose(Ub), Ub)
    tr = tf.linalg.trace(m)
    tr2 = tf.linalg.trace(m ** 2)
    trSq = tr ** 2
    g1 = tf.math.real(trSq) / 16.0
    g2 = tf.math.imag(trSq) / 16.0
    g3 = tf.math.real((trSq - tr2)) / 4.0
    return tf.concat([g1, g2, g3], 0)


def makhlinDistance(gs: tf.Tensor) -> tf.Tensor:
    roots = np.roots([1, -g3, 4 * np.sqrt(g1 ** 2 + g2 ** 2) - 1, g3 - 4 * g1]).real()
    roots = np.round(roots, 5)
    z = np.sort(roots)
    print("roots: ", roots)
    print("sorted: ", z)

    d = g3 * np.sqrt(g1 ** 2 + g2 ** 2) - g1
    s = np.pi - np.arccos(z[0]) - np.arccos(z[2])
    print("d: ", d, "s: ", s)
    if d>0 and s>0:
        return d
    elif d<0 and s<0:
        return -d
    else:
        return 0

sqrtSWAP = np.matrix([
    [1, 0, 0, 0],
    [0,(1+1j)/2.0,(1-1j)/2.0,0],
    [0,(1-1j)/2.0,(1+1j)/2.0,0],
    [0,0,0,1],
])
SWAP = np.matrix([
    [1,0,0,0],
    [0,0,1,0],
    [0,1,0,0],
    [0,0,0,1]
])
CNOT = np.matrix([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,0,1],
    [0,0,1,0]
])
UNITY = np.matrix([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0, 0, 0, 1]
])
gs = makhlinInvariants(tf.constant(UNITY, dtype=tf.complex128))
print(gs)
#dist = makhlinDistance(g1, g2, g3)
#print(dist)
'''